In [0]:
# Imports
from pyspark.sql.types import *
from pyspark.sql.functions import (
    col,
    length,
    trim,
    regexp_replace,
    regexp_extract,
    to_date,
    to_timestamp,
    when,
    lpad
)
from pyspark.sql import DataFrame


In [0]:
# path = "/FileStore/bronze/encounters"
# path = "/FileStore/bronze/patients"
# path = "/FileStore/bronze/providers"
path = "/FileStore/bronze/conditions"
# path = "/FileStore/bronze/organizations"

In [0]:
# Load the bronze Delta table
df = spark.read.format("delta").load(path)

# Alternatively, use the Hive table if registered
# encounters_bronze = spark.table("bronze_encounters")


In [0]:
# schema with data types and nullability
df.printSchema()

# inspect values and formatting
display(df.limit(10))


root
 |-- start: date (nullable = true)
 |-- stop: date (nullable = true)
 |-- patient: string (nullable = true)
 |-- encounter: string (nullable = true)
 |-- system: string (nullable = true)
 |-- code: long (nullable = true)
 |-- description: string (nullable = true)
 |-- ingestion_timestamp: timestamp (nullable = true)
 |-- source_file: string (nullable = true)



start,stop,patient,encounter,system,code,description,ingestion_timestamp,source_file
2015-05-29,2016-05-27,8a4869c4-4545-219f-2c65-fc44f98f3edf,4f7d84f3-8207-7c46-3291-3e5ed0bfe0df,SNOMED-CT,314529007,Medication review due (situation),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2021-10-14,2021-11-18,0e4068e0-4312-9a70-8f4d-46d1ddb9ae7c,96c9dae7-5534-9b62-ff62-f9070d16cccc,SNOMED-CT,314529007,Medication review due (situation),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2015-06-13,2015-11-27,8a4869c4-4545-219f-2c65-fc44f98f3edf,b7d27489-fe6c-bd22-a90c-789c1aa2a7ce,SNOMED-CT,65363002,Otitis media (disorder),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2016-10-06,2016-12-05,8a4869c4-4545-219f-2c65-fc44f98f3edf,23998ffd-5bca-e28e-915c-42763fd6e7f0,SNOMED-CT,110030002,Concussion injury of brain (disorder),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2016-10-06,2016-12-05,8a4869c4-4545-219f-2c65-fc44f98f3edf,23998ffd-5bca-e28e-915c-42763fd6e7f0,SNOMED-CT,62106007,Concussion with no loss of consciousness (disorder),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2016-11-25,2019-12-06,8a4869c4-4545-219f-2c65-fc44f98f3edf,1a82bcb4-4af2-1e09-5d17-1b9de6e50323,SNOMED-CT,314529007,Medication review due (situation),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2022-01-20,2022-01-20,0e4068e0-4312-9a70-8f4d-46d1ddb9ae7c,dabf85d7-5cdf-ff0b-1374-9b14f06893e5,SNOMED-CT,314529007,Medication review due (situation),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2006-02-19,2020-04-08,c0ec9264-5dda-fea6-ddea-edf27c0e846b,1d0b7aa1-cb2a-1b69-acb5-39aac257236c,SNOMED-CT,232353008,Perennial allergic rhinitis with seasonal variation (disorder),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2016-05-11,2016-05-11,c0ec9264-5dda-fea6-ddea-edf27c0e846b,3ec5e7c4-4362-eedf-aa7e-5bddcacba3c4,SNOMED-CT,314529007,Medication review due (situation),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv
2016-05-11,2016-05-26,c0ec9264-5dda-fea6-ddea-edf27c0e846b,3ec5e7c4-4362-eedf-aa7e-5bddcacba3c4,SNOMED-CT,66383009,Gingivitis (disorder),2025-06-16T02:50:32.469+0000,/FileStore/tables/conditions.csv


In [0]:
display(df.select([
    sum(col(c).isNull().cast("int")).alias(c + "_nulls") for c in df.columns
]))

start_nulls,stop_nulls,patient_nulls,encounter_nulls,system_nulls,code_nulls,description_nulls,ingestion_timestamp_nulls,source_file_nulls
0,27429,0,0,0,0,0,0,0


Encounters

In [0]:
# Distinct values of encounter class
df.select("encounterclass").distinct().show()

# Count by provider (check for skew)
df.groupBy("provider").count().orderBy("count", ascending=False).show(10)


+--------------+
|encounterclass|
+--------------+
|      wellness|
|    outpatient|
|     emergency|
|       virtual|
|     inpatient|
|          home|
|    urgentcare|
|       hospice|
|    ambulatory|
|           snf|
+--------------+

+--------------------+-----+
|            provider|count|
+--------------------+-----+
|b4fddf7d-8270-355...| 4586|
|960489e1-19d9-39b...| 3289|
|6a8867c7-d162-343...| 3034|
|b8ff6ff2-d278-336...| 2905|
|019d7b1f-8b17-3b6...| 2824|
|17e1fc8a-cfb2-37a...| 2820|
|90be9325-b802-306...| 2668|
|5f479285-fe2c-369...| 2571|
|90d0f098-88e8-3f4...| 2513|
|40f1fbee-320e-3a0...| 2393|
+--------------------+-----+
only showing top 10 rows



In [0]:
# Check how many rows have missing or malformed costs
display(df.filter(col("total_claim_cost").rlike("^[0-9.]+$") == False))

# Check for duplicate encounter IDs
display(df.groupBy("id").count().filter("count > 1"))

id,start,stop,patient,organization,provider,payer,encounterclass,code,description,base_encounter_cost,total_claim_cost,payer_coverage,reasoncode,reasondescription,ingestion_timestamp,source_file


id,count


Working on Fxn for Processing All Synthea Bronze Tables

In [0]:
# Set widgets for use in a Databricks job
dbutils.widgets.text("source", "")  # Leave blank for manual multi-table mode
dbutils.widgets.text("bronze_base_path", "/FileStore/bronze")
dbutils.widgets.text("silver_base_path", "/FileStore/silver")

# Retrieve widget values
source = dbutils.widgets.get("source")
bronze_base_path = dbutils.widgets.get("bronze_base_path")
silver_base_path = dbutils.widgets.get("silver_base_path")

# Default to all tables if source is blank
if source:
    sources_to_process = [source]
else:
    sources_to_process = ["patients", "encounters", "providers", "organizations", "conditions"]
    # sources_to_process = "encounters"


In [0]:
def silver_transform(df: DataFrame, source: str) -> DataFrame:
    """
    Applies source-specific transformation logic for Silver layer.
    
    Parameters:
        df (DataFrame): Input DataFrame from Bronze layer
        source (str): Source name (e.g., 'patients', 'encounters', etc.)
        
    Returns:
        DataFrame: Transformed DataFrame ready for Silver layer
    """
    
    if source == "patients":
        # Synthea adds digits to end of names — regex to trim
        df = df.withColumn("first", regexp_replace(trim(col("first")), r"\d+$", "")) \
               .withColumn("middle", regexp_replace(trim(col("middle")), r"\d+$", "")) \
               .withColumn("last", regexp_replace(trim(col("last")), r"\d+$", "")) \
               .withColumn("maiden", regexp_replace(trim(col("maiden")), r"\d+$", "")) \
               .withColumn("suffix", regexp_replace(trim(col("suffix")), r"\d+$", "")) \
               .withColumn("birthdate", to_date("birthdate")) \
               .withColumn("deathdate", to_date("deathdate"))

    elif source == "encounters":
        # Timestamp cast and text field cleanup
        df = df.withColumn("start", to_timestamp("start")) \
               .withColumn("stop", to_timestamp("stop")) \
               .withColumn("encounterclass", trim(col("encounterclass"))) \
               .withColumn("description", trim(col("description"))) \
               .withColumn("reasondescription", trim(col("reasondescription"))) \
               .withColumn("base_encounter_cost", col("base_encounter_cost").cast("double")) \
               .withColumn("total_claim_cost", col("total_claim_cost").cast("double")) \
               .withColumn("payer_coverage", col("payer_coverage").cast("double"))

    elif source == "providers":
        # Trim digits from names and clean location fields
        df = df.withColumn("name", regexp_replace(trim(col("name")), r"(\D+)\d+\s+(\D+)\d+", r"\1 \2")) \
               .withColumn("speciality", trim(col("speciality"))) \
               .withColumn("address", trim(col("address"))) \
               .withColumn("city", trim(col("city"))) \
               .withColumn("state", trim(col("state"))) \
               .withColumn("zip_clean",
                    when(length(col("zip")) < 5, lpad(col("zip").cast("string"), 5, "0"))
                    .otherwise(col("zip").cast("string").substr(1, 5)))

    elif source == "organizations":
        # Clean contact and location info, fix ZIPs and phone format
        df = df.withColumn("name", trim(col("name"))) \
               .withColumn("address", trim(col("address"))) \
               .withColumn("city", trim(col("city"))) \
               .withColumn("state", trim(col("state"))) \
               .withColumn("phone", trim(col("phone"))) \
               .withColumn("zip_cleaned",
                    when(length(col("zip")) < 5, lpad(col("zip").cast("string"), 5, "0"))
                    .otherwise(col("zip").cast("string").substr(1, 5))) \
               .withColumn("phone",
                    regexp_replace(col("phone"), r"[^0-9]", "").substr(1, 10)) \
               .withColumn("revenue", col("revenue").cast("double")) \
               .withColumn("utilization", col("utilization").cast("int"))

    elif source == "conditions":
        # Cast dates and extract condition type from description
        df = df.withColumn("start", to_date("start")) \
               .withColumn("stop", to_date("stop")) \
               .withColumn("description", trim(col("description"))) \
               .withColumn("condition_type", regexp_extract(col("description"), r"\((.*?)\)", 1))

    else:
        raise ValueError(f"Unknown source: {source}")

    return df


In [0]:
for src in sources_to_process:
    input_path = f"{bronze_base_path}/{src}"
    output_path = f"{silver_base_path}/{src}"
    
    print(f"🔄 Processing {src} from {input_path} → {output_path}")
    
    try:
        df_bronze = spark.read.format("delta").load(input_path)
        df_silver = silver_transform(df_bronze, src)
        df_silver.write.format("delta").mode("overwrite").save(output_path)
        print(f"✅ Successfully wrote Silver table: {src}")
    except Exception as e:
        print(f"❌ Error processing {src}: {e}")


🔄 Processing patients from /FileStore/bronze/patients → /FileStore/silver/patients
✅ Successfully wrote Silver table: patients
🔄 Processing encounters from /FileStore/bronze/encounters → /FileStore/silver/encounters
✅ Successfully wrote Silver table: encounters
🔄 Processing providers from /FileStore/bronze/providers → /FileStore/silver/providers
✅ Successfully wrote Silver table: providers
🔄 Processing organizations from /FileStore/bronze/organizations → /FileStore/silver/organizations
✅ Successfully wrote Silver table: organizations
🔄 Processing conditions from /FileStore/bronze/conditions → /FileStore/silver/conditions
✅ Successfully wrote Silver table: conditions
